In [5]:
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# Lê a data de corte da tabela de controle
data_base = spark.sql("""
    SELECT vDataCorte FROM lk_ControleProcessos.tab_controle_incremental
""").collect()[0]['vDataCorte']

# Converte para datetime.date se necessário
if isinstance(data_base, str):
    data_base = datetime.strptime(data_base, "%Y-%m-%d").date()

# Força o dia para 1º do mês
data_inicio_mes = data_base.replace(day=1)

# Formata como string para o SQL
data_limite_str = data_inicio_mes.strftime("%Y-%m-%d")  # Exemplo: '2025-01-01'

# Configurações para leitura e escrita de datas antigas (parquet)
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# Atualiza datas inválidas (antes de 1900-01-01) para uma data padrão
spark.sql("""
    UPDATE pesocialeventos
    SET RECCREATEDON = DATE('2000-01-01')
    WHERE RECCREATEDON IS NOT NULL
      AND RECCREATEDON < DATE('1900-01-01')
""")

# Verifica os limites após a atualização
spark.sql("""
    SELECT MIN(RECCREATEDON) AS DATA_MINIMA, MAX(RECCREATEDON) AS DATA_MAXIMA
    FROM pesocialeventos
""").show()

# Deleta registros com RECCREATEDON a partir da data de corte
spark.sql(f"""
    DELETE FROM pesocialeventos
    WHERE RECCREATEDON >= DATE('{data_limite_str}')
""")

StatementMeta(, c491d98d-4798-406d-80a2-793d090819d3, 7, Finished, Available, Finished)

+-------------------+-------------------+
|        DATA_MINIMA|        DATA_MAXIMA|
+-------------------+-------------------+
|2018-09-28 16:24:52|2025-07-24 10:27:16|
+-------------------+-------------------+



DataFrame[num_affected_rows: bigint]